# Quick Start
Federated Learning (FL) is a learning paradigm for collaboratively learning models from isolated data without directly sharing privacy information, which helps to satisfy the requirements of privacy protection of the public. **FederatedScope**, a comprehensive platform for FL based on a message-oriented framework, aims to provide easy-used and flexible support for developers who want to quick start and custom task-specific FL procedures.

We first provide an end-to-end standalone example to illustrate how to implement a standard FL task with **FederatedScope**.

## Configurations
**FederatedScope** organize the configuration through `yacs.config.cfgNode` and can be found in `federatedscope.configs`. Please refer to the [official documentation](https://federatedscope.io/refs/index) for specific instructions on how to configure `cfg`.

In [2]:
from federatedscope.core.configs.config import global_cfg

cfg = global_cfg.clone()
print(cfg)

asyn:
  min_received_num: 2
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: 
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: toy
device: -1
distribute:
  client_host: 0.0.0.0
  client_port: 50050
  data_file: data
  data_idx: -1
  grpc_enable_http_proxy: False
  grpc_max_recei

## 1. Prepare datasets
To run an FL task, firstly you should prepare datasets for FL. The DataZoo provided in **FederatedScope** can help to automatically download and preprocess widely-used public datasets from various FL applications, including computer vision, natural language processing, graph learning, recommendation, etc. Developers can conveniently conduct experiments on the provided dataset via specifying `cfg.data.type = DATASET_NAME` in the configuration. 
We also support developers to adopt custom datasets, please refer to [https://federatedscope.io/docs/own-case/]() for more details about the provided datasets in DataZoo, and refer to [02. Start your own case](./02_start_your_own_case.ipynb) for introducing custom datasets in **FederatedScope**.

In [3]:
from federatedscope.core.auxiliaries.data_builder import get_data

cfg.data.type = 'femnist'
cfg.data.splits = [0.6, 0.2, 0.2]
cfg.data.batch_size = 10
cfg.data.subsample = 0.05
cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.1307], 'std': [0.3081]}]]

data, modified_cfg = get_data(cfg.clone())
cfg.merge_from_other_cfg(modified_cfg)

Preprocess data (Please leave enough space)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.74s/it]


## 2. Prepare models
You should specify the model architecture that will be federally trained, such as ConvNet or LSTM. FederatedScope includes the ModelZoo to provide the implementation of famous model architectures for various FL applications. Developers can set up `cfg.model.type = MODEL_NAME` to apply a specific model architecture in FL tasks. We allow developers to use custom models via registering without caring about the federated process. You can refer to [02. Start your own case](./02_start_your_own_case.ipynb) for more details about how to customize a model architecture.

In [4]:
cfg.model.type = 'convnet2'
cfg.model.out_channels = 62

##  3. Start running an FL task (standalone)
Note that FederatedScope provides a unified interface for both standalone mode and distributed mode, and allows users to change via configuring. 

### Task-specific configuration.
Here we demonstrate how to run a standard FL task with FederatedScope as an example, with setting `cfg.data.type = 'FEMNIST'`and `cfg.model.type = 'ConvNet2'` to run vanilla FedAvg for an image classification task. Now configure the Federated Learning process with `cfg`.

In [5]:
cfg.use_gpu = False
cfg.eval.freq = 10
cfg.eval.metrics = ['acc', 'loss_regular']

cfg.federate.mode = 'standalone'
cfg.federate.local_update_steps = 5
cfg.federate.total_round_num = 20
cfg.federate.sample_client_num = 5
cfg.federate.client_num = 10

cfg.train.optimizer.lr = 0.001
cfg.train.optimizer.weight_decay = 0.0
cfg.grad.grad_clip = 5.0

cfg.criterion.type = 'CrossEntropyLoss'
cfg.trainer.type = 'cvtrainer'
cfg.seed = 123

### Check `cfg` and set up `logger`
We now need to check if the `cfg` has conflicts with default `cfg` and set up `logger` to monitor. The logs will be output in the Cell below.

In [9]:
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger

setup_seed(cfg.seed)
update_logger(cfg)


2022-08-29 22:02:40,106 (utils:129)INFO: the current machine is at 192.168.101.227
2022-08-29 22:02:40,107 (utils:131)INFO: the current dir is F:\python\FederatedScope
2022-08-29 22:02:40,107 (utils:132)INFO: the output dir is exp\FedAvg_convnet2_on_femnist_lr0.001_lstep1_\FedAvg_convnet2_on_femnist_lr0.001_lstep1__


### Ready to go, let's get started!

In [7]:
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls

Fed_runner = FedRunner(data=data,
                       server_class=get_server_cls(cfg),
                       client_class=get_client_cls(cfg),
                       config=cfg.clone())
Fed_runner.run()

2022-08-29 21:56:08,126 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-08-29 21:56:08,152 (fed_runner:302)INFO: Client 1 has been set up ... 
2022-08-29 21:56:08,166 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-08-29 21:56:08,177 (fed_runner:302)INFO: Client 3 has been set up ... 
2022-08-29 21:56:08,190 (fed_runner:302)INFO: Client 4 has been set up ... 
2022-08-29 21:56:08,202 (fed_runner:302)INFO: Client 5 has been set up ... 
2022-08-29 21:56:08,214 (fed_runner:302)INFO: Client 6 has been set up ... 
2022-08-29 21:56:08,225 (fed_runner:302)INFO: Client 7 has been set up ... 
2022-08-29 21:56:08,236 (fed_runner:302)INFO: Client 8 has been set up ... 
2022-08-29 21:56:08,250 (fed_runner:302)INFO: Client 9 has been set up ... 
2022-08-29 21:56:08,259 (fed_runner:302)INFO: Client 10 has been set up ... 
2022-08-29 21:56:08,260 (trainer:324)INFO: Model meta-info: <class 'federatedscope.cv.model.cnn.ConvNet2'>.
2022-08-29 21:56:08,262 (trainer:332)INFO: Num of orig

2022-08-29 21:56:09,728 (server:321)INFO: ----------- Starting a new training round (Round #4) -------------
2022-08-29 21:56:09,754 (client:259)INFO: {'Role': 'Client #7', 'Round': 4, 'Results_raw': {'train_loss': 43.128381, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.312838}}
2022-08-29 21:56:09,780 (client:259)INFO: {'Role': 'Client #9', 'Round': 4, 'Results_raw': {'train_loss': 45.318069, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.531807}}
2022-08-29 21:56:09,807 (client:259)INFO: {'Role': 'Client #3', 'Round': 4, 'Results_raw': {'train_loss': 40.69881, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.069881}}
Unsupported operator aten::add_ encountered 2 time(s)
Unsupported operator aten::max_pool2d encountered 2 time(s)
2022-08-29 21:56:09,868 (client:259)INFO: {'Role': 'Client #10', 'Round': 4, 'Results_raw': {'train_loss': 42.67467, 'train_total': 10, 'train_loss_

2022-08-29 21:56:11,825 (server:487)INFO: {'Role': 'Server #', 'Round': 10, 'Results_weighted_avg': {'test_loss': 276.977604, 'test_total': 66.4, 'test_loss_regular': 0.0, 'test_acc': 0.013554, 'test_avg_loss': 4.105541, 'val_loss': 275.39893, 'val_total': 65.9, 'val_loss_regular': 0.0, 'val_acc': 0.00607, 'val_avg_loss': 4.115098}, 'Results_avg': {'test_loss': 272.607926, 'test_total': 66.4, 'test_loss_regular': 0.0, 'test_acc': 0.013773, 'test_avg_loss': 4.105667, 'val_loss': 271.184964, 'val_total': 65.9, 'val_loss_regular': 0.0, 'val_acc': 0.005739, 'val_avg_loss': 4.116175}, 'Results_fairness': {'test_total': 66.4, 'val_total': 65.9, 'test_loss_std': 34.512633, 'test_loss_bottom_decile': 236.176417, 'test_loss_top_decile': 327.255363, 'test_loss_regular_std': 0.0, 'test_loss_regular_bottom_decile': 0.0, 'test_loss_regular_top_decile': 0.0, 'test_acc_std': 0.019376, 'test_acc_bottom_decile': 0.0, 'test_acc_top_decile': 0.064516, 'test_avg_loss_std': 0.025563, 'test_avg_loss_bottom_

2022-08-29 21:56:12,824 (client:259)INFO: {'Role': 'Client #8', 'Round': 16, 'Results_raw': {'train_loss': 43.896275, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.389627}}
2022-08-29 21:56:12,837 (server:321)INFO: ----------- Starting a new training round (Round #17) -------------
2022-08-29 21:56:12,862 (client:259)INFO: {'Role': 'Client #8', 'Round': 17, 'Results_raw': {'train_loss': 40.803576, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.080358}}
2022-08-29 21:56:12,891 (client:259)INFO: {'Role': 'Client #3', 'Round': 17, 'Results_raw': {'train_loss': 41.626368, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.162637}}
2022-08-29 21:56:12,915 (client:259)INFO: {'Role': 'Client #4', 'Round': 17, 'Results_raw': {'train_loss': 42.393842, 'train_total': 10, 'train_loss_regular': 0.0, 'train_acc': 0.0, 'train_avg_loss': 4.239384}}
2022-08-29 21:56:12,945 (client:259)INFO: {'Ro

2022-08-29 21:56:14,198 (server:437)INFO: {'Role': 'Client #5', 'Round': 20, 'Results_raw': {'test_loss': 277.36944, 'test_total': 67, 'test_loss_regular': 0.0, 'test_acc': 0.0, 'test_avg_loss': 4.139842, 'val_loss': 274.444575, 'val_total': 67, 'val_loss_regular': 0.0, 'val_acc': 0.014925, 'val_avg_loss': 4.096188}}
2022-08-29 21:56:14,200 (server:437)INFO: {'Role': 'Client #6', 'Round': 20, 'Results_raw': {'test_loss': 205.668097, 'test_total': 50, 'test_loss_regular': 0.0, 'test_acc': 0.0, 'test_avg_loss': 4.113362, 'val_loss': 206.598344, 'val_total': 50, 'val_loss_regular': 0.0, 'val_acc': 0.02, 'val_avg_loss': 4.131967}}
2022-08-29 21:56:14,202 (server:437)INFO: {'Role': 'Client #7', 'Round': 20, 'Results_raw': {'test_loss': 290.600082, 'test_total': 71, 'test_loss_regular': 0.0, 'test_acc': 0.028169, 'test_avg_loss': 4.092959, 'val_loss': 284.35442, 'val_total': 70, 'val_loss_regular': 0.0, 'val_acc': 0.071429, 'val_avg_loss': 4.062206}}
2022-08-29 21:56:14,204 (server:437)INFO:

{'client_best_individual': {'val_loss_regular': 0.0,
  'val_loss': 206.59834384918213,
  'test_loss': 205.6680965423584,
  'test_total': 50.0,
  'test_loss_regular': 0.0,
  'test_acc': 0.0,
  'test_avg_loss': 4.042709681295579,
  'val_total': 50.0,
  'val_acc': 0.0,
  'val_avg_loss': 4.062205995832171},
 'client_summarized_weighted_avg': {'val_loss_regular': 0.0,
  'val_loss': 274.65778040849744,
  'test_loss': 276.35139922600194,
  'test_total': 66.4,
  'test_loss_regular': 0.0,
  'test_acc': 0.010542168674698794,
  'test_avg_loss': 4.096276154360139,
  'val_total': 65.9,
  'val_acc': 0.012139605462822459,
  'val_avg_loss': 4.103880867791646},
 'client_summarized_avg': {'val_loss_regular': 0.0,
  'val_loss': 270.44574918746946,
  'test_loss': 271.99273664951323,
  'test_total': 66.4,
  'test_loss_regular': 0.0,
  'test_acc': 0.009985360189812711,
  'test_avg_loss': 4.096450669002026,
  'val_total': 65.9,
  'val_acc': 0.011901217241100106,
  'val_avg_loss': 4.104852310339757},
 'client

Congratulations, you've learned how to run a complete FL process. Next you can modify `cfg` to try out different built-in features in **FederatedScope**, or you can continue to learn [02. Start your own case](./02_start_your_own_case.ipynb).